In [ ]:
import os
import shutil
from tqdm import tqdm

def collect_zips(base_path, folder_name, target_folder_name="raw_zips"):
    target_dir = os.path.join(base_path, target_folder_name)
    data_path = os.path.join(base_path,folder_name)
    os.makedirs(target_dir, exist_ok=True)
    
    zip_files = []
    # 1. 모든 하위 폴더를 순회하며 zip 파일 목록 수집
    for root, dirs, files in os.walk(data_path):
        # 타겟 폴더 자체는 검색에서 제외
        if target_folder_name in root:
            continue
            
        for file in files:
            if file.endswith(".zip"):
                zip_files.append(os.path.join(root, file))
    
    print(f"📦 총 {len(zip_files)}개의 zip 파일을 찾았습니다.")
    
    # 2. 파일 이동
    for file_path in tqdm(zip_files, desc="Moving files"):
        file_name = os.path.basename(file_path)
        dest_path = os.path.join(target_dir, file_name)
        
        # 파일 이름 충돌 방지 (중복 이름이 있을 경우)
        if os.path.exists(dest_path):
            name, ext = os.path.splitext(file_name)
            dest_path = os.path.join(target_dir, f"{name}_dup{ext}")
            
        shutil.move(file_path, dest_path)

# 실행
base_path = "./DataSets/AIHub"
folder_list = ["030.웹데이터_기반_한국어_말뭉치_데이터", "121.한국어_성능이_개선된_초거대AI_언어모델_개발_및_데이터"]
for folder_name in folder_list:
    collect_zips(base_path, folder_name)
print("✅ 모든 zip 파일이 raw_zips 폴더로 모였습니다.")

📦 총 4개의 zip 파일을 찾았습니다.


Moving files: 100%|██████████| 4/4 [00:00<00:00, 19831.22it/s]


📦 총 55개의 zip 파일을 찾았습니다.


Moving files: 100%|██████████| 55/55 [00:00<00:00, 54113.70it/s]

✅ 모든 zip 파일이 raw_zips 폴더로 모였습니다.


In [ ]:
import os
import zipfile
from tqdm import tqdm
def smart_extract(zip_path, target_dir):
    try:
        os.makedirs(target_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file in zip_ref.namelist():
                # [핵심 수정] 파일명 앞의 슬래시(/)나 백슬래시(\)를 제거하여 상대 경로로 만듦
                clean_name = file.lstrip('/') 
                
                # 한글 깨짐 방지 처리
                try:
                    decoded_name = clean_name.encode('cp437').decode('cp949')
                except:
                    decoded_name = clean_name
                
                target_path = os.path.join(target_dir, decoded_name)
                
                if file.endswith('/'):
                    os.makedirs(target_path, exist_ok=True)
                    continue
                
                # 추출될 폴더 생성 확인
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                
                # 파일 쓰기
                with zip_ref.open(file) as source, open(target_path, "wb") as target:
                    target.write(source.read())
                    
    except Exception as e:
        print(f"❌ {os.path.basename(zip_path)} 해제 실패: {e}")

# 경로 설정
base_path = "./DataSets/AIHub"
zip_folder = os.path.join(base_path, "kor_xlarge_raw_zips") # zip 파일들이 모여있는 곳
extract_base = os.path.join(base_path, "kor_xlarge_extracted")

# 1. 대상 폴더 생성
os.makedirs(extract_base, exist_ok=True)

# 2. 모든 zip 파일 찾아 해제
zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]
print(f"📦 총 {len(zip_files)}개의 압축 파일을 찾았습니다.")
corpus_624_name_list = ["TL1.zip", "TS1.zip", "VL1.zip", "VS1.zip"]
for z_file in tqdm(zip_files, desc="Extracting"):
    # 파일명에 따라 하위 폴더 분류 (624 또는 71748 키워드로 구분)
    sub_folder = "corpus_624" if z_file in corpus_624_name_list else "inst_71748"
    target_path = os.path.join(extract_base, sub_folder)
    
    smart_extract(os.path.join(zip_folder, z_file), target_path)

print("✅ 모든 압축 해제가 완료되었습니다!")

📦 총 59개의 압축 파일을 찾았습니다.


Extracting: 100%|██████████| 59/59 [05:13<00:00,  5.32s/it]

✅ 모든 압축 해제가 완료되었습니다!


In [ ]:
import json
import glob

# 추출된 폴더 내의 JSON 파일 하나 찾기
json_files = glob.glob("./DataSets/AIHub/kor_xlarge_extracted/**/*.json", recursive=True)

if json_files:
    with open(json_files[0], 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
        # 키(Key) 구조 출력
        print("JSON 주요 키:", sample_data.keys())
        # 데이터의 아주 일부만 출력하여 본문이 어디 있는지 확인
        print(json.dumps(sample_data, indent=2, ensure_ascii=False)[:500])
else:
    print("❌ JSON 파일을 찾을 수 없습니다. 경로를 다시 확인하세요.")

JSON 주요 키: dict_keys(['header', 'named_entity'])
{
  "header": {
    "identifier": "텍스트_웹데이터 말뭉치 구축__7000018002",
    "name": "컨테이너 텍스트 인식을 위한 학습용 데이터셋",
    "category": "0",
    "type": "0",
    "source_file": "BWCU217000018002",
    "source": "0",
    "subject": "CU"
  },
  "named_entity": [
    {
      "title": [
        {
          "sentence": "빅톤 (이름), 비현실적 피지컬..훈훈함 한도 초과",
          "labels": []
        }
      ],
      "subtitle": [],
      "content": [
        {
          "sentence": "그룹 빅톤 멤버 (이름)가 남다른 훈훈함을 뽐냈다..",
          "labels":


In [ ]:
import json
import os
import glob
from tqdm import tqdm

def extract_and_merge(base_dir, output_file):
    json_files = glob.glob(os.path.join(base_dir, "**/*.json"), recursive=True)
    print(f"📄 총 {len(json_files)}개 파일에서 'named_entity' 구조 추출 시작...")

    with open(output_file, 'w', encoding='utf-8') as f_out:
        for file_path in tqdm(json_files):
            try:
                with open(file_path, 'r', encoding='utf-8') as f_in:
                    data = json.load(f_in)
                
                texts = []

                # 핵심: named_entity 리스트를 순회
                if 'named_entity' in data:
                    for entity in data['named_entity']:
                        # 1. title 필드에서 sentence 추출
                        for section in ['title', 'content', 'subtitle']:
                            if section in entity:
                                for item in entity[section]:
                                    if 'sentence' in item:
                                        texts.append(item['sentence'])
                
                # 만약 위 구조가 아니고 다른 데이터셋 섞여있을 경우 대비
                elif 'document' in data:
                    for doc in data['document']:
                        for key in ['paragraph', 'utterance']:
                            if key in doc:
                                for p in doc[key]:
                                    if 'form' in p: texts.append(p['form'])

                # 파일에 쓰기
                clean_texts = [t.strip() for t in texts if t and len(t.strip()) > 0]
                if clean_texts:
                    f_out.write("\n".join(clean_texts) + "\n")

            except Exception:
                continue

# 경로 설정
extract_path = "./DataSets/AIHub/kor_xlarge_extracted"
total_txt = "./DataSets/AIHub/total_corpus.txt"

if os.path.exists(total_txt): os.remove(total_txt)

extract_and_merge(extract_path, total_txt)
print(f"✅ 통합 완료! 생성된 파일: {total_txt}")

if os.path.exists(total_txt):
    size_mb = os.path.getsize(total_txt) / (1024**2)
    print(f"✅ 생성 완료! 파일 크기: {size_mb:.2f} MB")

📄 총 63786개 파일에서 'named_entity' 구조 추출 시작...


100%|██████████| 63786/63786 [06:42<00:00, 158.38it/s] 

✅ 통합 완료! 생성된 파일: /media/edint/64d115f7-57cc-417b-acf0-7738ac091615/Ivern/DataSets/AIHub/total_corpus.txt
✅ 생성 완료! 파일 크기: 11293.19 MB
